In [1]:
import warnings
warnings.filterwarnings('ignore')

# Exploration

- App: The application name.
- Category: The category the app belongs to.
- Rating: Overall user rating of the app.
- Reviews: Number of user reviews for the app.
- Size: The size of the app.
- Installs: Number of user installs for the app.
- Type: Either "Paid" or "Free".
- Price: The price of the app.

- Content Rating: The age group the app is targeted at - "Children" / "Mature 21+" / "Adult".
- Genres: Possibly multiple genres the app belongs to.
- Last Updated: The date the app was last updated.
- Current Ver: The current version of the app.
- Android Ver: The Android version is needed for this app.

In [2]:
import pandas as pd
import plotly.graph_objects as go
from utils import data
from dateutil.parser import parse

In [3]:
data.head()

,Unnamed: 0,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,7-Jan-18,1.0.0,4.0.3 and up
1,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.0.0,4.0.3 and up
2,2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,1-Aug-18,1.2.4,4.0.3 and up
3,3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,8-Jun-18,Varies with device,4.2 and up
4,4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up


In [4]:
# On remarque que la première colonne n'a pas de nom.
data.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
data.columns

Index(['id', 'App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs',
       'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated',
       'Current Ver', 'Android Ver'],
      dtype='object')

In [5]:
data.shape

(10841, 14)

## Verification de l'integrité des données

In [6]:
# Est-ce qu'il y a des nulls ?
data.isnull().sum()

id                   0
App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [7]:
# Combien de valeurs uniques par colonne ?
for col in data.columns:
    print(f"{col}:")
    unique = data[col].unique()
    unique_length = unique.size
    print(f"\t Unique count: {unique_length}")
    # Et pour ceux avoir moins de 50 valeurs on les affiches
    if unique_length > 50:
        print("\t Too Many Values")
    else:
        print(f"\t {unique}")
    print()
    

id:
	 Unique count: 10841
	 Too Many Values

App:
	 Unique count: 9660
	 Too Many Values

Category:
	 Unique count: 34
	 ['ART_AND_DESIGN' 'AUTO_AND_VEHICLES' 'BEAUTY' 'BOOKS_AND_REFERENCE'
 'BUSINESS' 'COMICS' 'COMMUNICATION' 'DATING' 'EDUCATION' 'ENTERTAINMENT'
 'EVENTS' 'FINANCE' 'FOOD_AND_DRINK' 'HEALTH_AND_FITNESS' 'HOUSE_AND_HOME'
 'LIBRARIES_AND_DEMO' 'LIFESTYLE' 'GAME' 'FAMILY' 'MEDICAL' 'SOCIAL'
 'SHOPPING' 'PHOTOGRAPHY' 'SPORTS' 'TRAVEL_AND_LOCAL' 'TOOLS'
 'PERSONALIZATION' 'PRODUCTIVITY' 'PARENTING' 'WEATHER' 'VIDEO_PLAYERS'
 'NEWS_AND_MAGAZINES' 'MAPS_AND_NAVIGATION' '1.9']

Rating:
	 Unique count: 41
	 [ 4.1  3.9  4.7  4.5  4.3  4.4  3.8  4.2  4.6  3.2  4.   nan  4.8  4.9
  3.6  3.7  3.3  3.4  3.5  3.1  5.   2.6  3.   1.9  2.5  2.8  2.7  1.
  2.9  2.3  2.2  1.7  2.   1.8  2.4  1.6  2.1  1.4  1.5  1.2 19. ]

Reviews:
	 Unique count: 6002
	 Too Many Values

Size:
	 Unique count: 462
	 Too Many Values

Installs:
	 Unique count: 22
	 ['10,000+' '500,000+' '5,000,000+' '50,000,

On constate qu'une ligne a ses colonnes est décalée


In [8]:
data[data['Installs'] == 'Free']

,id,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,11-Feb-18,1.0.19,4.0 and up,NaN


---

# Nettoyage des données

Suppression de la ligne dont les colonnes sont décalées

In [9]:
data = data[data['Installs'] != 'Free']

Transformation de la colonne "Last Updated" en date

In [10]:
data["Last Updated"] = data["Last Updated"].apply(lambda x: parse(x))
data.dtypes

id                         int64
App                       object
Category                  object
Rating                   float64
Reviews                   object
Size                      object
Installs                  object
Type                      object
Price                     object
Content Rating            object
Genres                    object
Last Updated      datetime64[ns]
Current Ver               object
Android Ver               object
dtype: object

Créer une colonnes avec le nombre d'installation en nombre

In [11]:
data['Installs'] = data['Installs'].str.replace('[+,]', '', regex=True).astype(int)
data['Installs'].unique()

array([     10000,     500000,    5000000,   50000000,     100000,
            50000,    1000000,   10000000,       5000,  100000000,
       1000000000,       1000,  500000000,         50,        100,
              500,         10,          1,          5,          0])

In [12]:
data['Price'] = data['Price'].str.replace('[$]', '', regex=True).astype(float)

Cast du type de Reviews en int

In [13]:
data['Reviews'] = data['Reviews'].map(int)

Ajout d'une colonne représentant le nombre d'utilisateurs n'ayant pas attribué de note.

In [14]:
data['Unreviewed'] = data['Installs'] - data['Reviews']
data['Unreviewed'].size

10840

---

# Graphes

## Moyenne du nombre de téléchargements par catégorie d'applications

Préparation des données

In [15]:
# Calculer le nombre total de téléchargements par catégorie
total_downloads_by_category = data.groupby('Category')['Installs'].sum().sort_values()
# Calculer le nombre total d'applications par catégorie
total_apps_by_category = data['Category'].value_counts()
# Calculer la moyenne des téléchargements par catégorie
average_downloads_by_category = total_downloads_by_category / total_apps_by_category
# Trier les catégories d'applications par ordre croissant de moyenne de téléchargements
average_downloads_by_category = average_downloads_by_category.sort_values(ascending=True)

Création du graphe

In [34]:
fig = go.Figure(data=[go.Bar(
    x=average_downloads_by_category.index,
    y=average_downloads_by_category
)])
fig.update_layout(
    title_text='Moyenne du nombre de téléchargements par catégorie d\'applications',
    xaxis_title="Catégorie", 
    yaxis_title="Moyenne du nombre de téléchargements", 
    width=900,
    xaxis=dict(tickmode = 'linear', dtick = 1),
    title_x=0.5
)
;

''

In [17]:
fig.show()

# Nombre d'application gratuite et payante

Préparation des données

In [18]:
types = data["Type"]
type_counts = types.value_counts()

Création du graphe

In [19]:
fig = go.Figure(data=[go.Pie(
    labels=type_counts.index,
    values=type_counts
)])
fig.update_layout(
    title_text='Nombre d\'application gratuite et payante',
    width=500,
    title_x=0.5
)

In [20]:

fig.show()

# Nombre d'applications par catégorie

Obtention du nombre d'applications par catégorie

In [21]:
category = data['Category'].value_counts()

Affichage du nombre d'applications par catégorie dans un graphique à barres horizontales

In [22]:
fig = go.Figure(data=[go.Bar(
    y=category.index,
    x=category,
    orientation='h'
)])
fig.update_layout(
    title_text='Nombre d\'applications par catégorie',
    xaxis_title="Nombre d'applications", 
    width=900,
    title_x=0.5
)

In [23]:
fig.show()

# Meilleures catégories par note sur le contenu

Préparation des données

In [24]:
# Groupe les valeurs par note de sur le contenu
grouped_categories = data.groupby('Content Rating')['App'].count()
top_categories = grouped_categories.sort_values(ascending=False)

Création du graphe

In [25]:
fig = go.Figure(data=[go.Pie(
    labels=top_categories.index,
    values=top_categories
)])
fig.update_layout(
    title_text='Meilleures catégories par note sur le contenu',
    width=500,
    title_x=0.5
)
fig.show()

# Nombre d\'application par note

In [26]:
ratings_count = data['Rating'].value_counts()

In [27]:
fig = go.Figure(data=[go.Bar(
    x=ratings_count.index,
    y=ratings_count
)])
fig.update_layout(
    title_text='Nombre d\'application par note',
    xaxis_title="Note", 
    yaxis_title="Nombre d'application à cette note", 
    width=900,
    xaxis=dict(tickmode = 'linear', dtick = 1),
    title_x=0.5
)

In [28]:
fig.show() 

# Pourcentage d'avis posté comparé au nombre d'installations

In [29]:
# Groupe les valeurs par note de sur le contenu
reviews_and_unreviewed = data[['Reviews', 'Unreviewed']].sum()

In [30]:
fig = go.Figure(data=[go.Pie(
    labels=reviews_and_unreviewed.index,
    values=reviews_and_unreviewed,
    
)])
fig.update_layout(
    title_text='Pourcentage d\'avis posté comparé au<br>nombre d\'installations',
    width=500,
    title_x=0.5
)

In [31]:
fig.show()